这篇博客（[视频](https://www.bilibili.com/video/BV1r3411979W/?spm_id_from=333.999.0.0&vd_source=b4496ba547455f3db613a8f8de6bea01)）记录区块链技术投资公告效应研究样本的数据清洗过程，主要内容是剔除并非区块链技术投资事件相关公告；剔除 ST 类公司；剔除年度、季度公告、发债类公告、招股说明书、发行股票募集说明书、非公开发行普通股、非公开发行优先股、发行保荐书、社会责任公告等。
<!-- TEASER_END -->

In [1]:
import numpy as np
import pandas as pd

In [2]:
raw_data = pd.read_csv('./data/raw_data.csv')

In [3]:
raw_data.head()

,secCode,secName,orgId,announcementTitle,announcementTime,adjunctUrl,announcementContent
0,000631,顺发恒业,gssz0000631,*ST 兰宝：重大资产出售、发行股份购买资产暨关联交易报告书（修订稿）,2009-02-28,http://static.cninfo.com.cn/finalpage/2009-02-...,（二）公司业绩持续下滑，且无改善迹象 由于个别银行发现公司资金链紧张，逐步停止转贷并收缩银根...
1,600596,新安股份,gssh0600596,新安股份：2008年年度报告,2009-03-10,http://static.cninfo.com.cn/finalpage/2009-03-...,3.新安包装公司搬迁事项 因新安江桥东区块旧城改造，新安包装公司于本期从老厂区（新安江街道桥...
2,600267,海正药业,gssh0600267,海正药业：2008年年度股东大会会议资料,2009-03-20,http://static.cninfo.com.cn/finalpage/2009-03-...,"100.00% 141,278 100.00% 115,132 100.00% 公司凭借原料..."
3,002244,滨江集团,9900004730,滨江集团：2008年年度报告,2009-03-20,http://static.cninfo.com.cn/finalpage/2009-03-...,江干科技经济园区地块开发协议书》（江科园协字【2006】035 号），约定就“S08、09、...
4,000301,东方盛虹,gssz0000301,东方市场：2008年年度报告,2009-03-24,http://static.cninfo.com.cn/finalpage/2009-03-...,公司将采取措施进一步完善产业链，继续减少关联交易。


In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11050 entries, 0 to 11049
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   secCode              11050 non-null  object
 1   secName              11050 non-null  object
 2   orgId                11050 non-null  object
 3   announcementTitle    11050 non-null  object
 4   announcementTime     11050 non-null  object
 5   adjunctUrl           11050 non-null  object
 6   announcementContent  11050 non-null  object
dtypes: object(7)
memory usage: 604.4+ KB


In [6]:
# 剔除 announcement_content 中不含 “区块链” 关键词的样本
def filter_content(c):
    if c.find('区块链') == -1:
        return False
    return True

In [7]:
raw_data.loc[:,'announcementContent'].apply(lambda c:filter_content(c))

0        False
1        False
2        False
3        False
4        False
         ...  
11045    False
11046    False
11047    False
11048    False
11049     True
Name: announcementContent, Length: 11050, dtype: bool

In [8]:
data1 = raw_data.loc[raw_data.loc[:,'announcementContent'].apply(lambda c:filter_content(c)), :]

In [9]:
len(data1)

7196

In [10]:
# 剔除 ST 类公司
def filter_ST(t):
    if t.find('ST') == -1:
        return True
    return False

In [11]:
data2 = data1.loc[data1.loc[:,'secName'].apply(lambda t: filter_ST(t)), :]

In [12]:
len(data2)

6937

In [13]:
# 剔除年度、季度公告、发债类公告、招股说明书、发行股票募集说明书、非公开发行普通股、非公开发行优先股、发行保荐书、社会责任公告
def filter_title(t):
    if any([t.find('年度')!=-1, t.find('季度')!=-1, t.find('债券')!=-1, t.find('公开发行股票')!=-1, t.find('发行股票募集')!=-1,
            t.find('非公开发行')!=-1, t.find('发行保荐书')!=-1, t.find('社会责任')!=-1, t.find('发行')!=-1]):
        return False
    return True

In [14]:
data3 = data2.loc[data2.loc[:,'announcementTitle'].apply(lambda t:filter_title(t)), :]

In [15]:
len(data3)

1348

In [16]:
# 筛选首份公告
final = data3.drop_duplicates(subset=['secCode'], keep='first')

In [17]:
len(final)

394

In [18]:
final.head()

,secCode,secName,orgId,announcementTitle,announcementTime,adjunctUrl,announcementContent
1150,002152,广电运通,9900003423,广电运通：2016年4月1日投资者关系活动记录表,2016-04-06,http://static.cninfo.com.cn/finalpage/2016-04-...,9、公司提过有储备区块链技术，是否可以介绍？
1174,002177,御银股份,9900003781,御银股份：2016年4月21日投资者关系活动记录表,2016-04-21,http://static.cninfo.com.cn/finalpage/2016-04-...,4、公司在区块链技术上未来有什么规划？
1196,300386,飞天诚信,9900023058,飞天诚信：2016年4月22日投资者关系活动记录表,2016-04-25,http://static.cninfo.com.cn/finalpage/2016-04-...,公司在区块链部分如何布局?
1255,000997,新 大 陆,gssz0000997,新 大 陆：董事会关于本次募集资金使用的可行性分析,2016-06-03,http://static.cninfo.com.cn/finalpage/2016-06-...,支付行业技术创新和业务创新不断涌现，包括 NFC、条码（二维码）支付、可信服务平台（TSM）...
1266,300423,鲁亿通,9900028812,鲁亿通：关于本次交易前12个月内购买、出售资产的说明,2016-06-09,http://static.cninfo.com.cn/finalpage/2016-06-...,同时，上市公司拟采用定价发行的方式向纪法清、孔剑平、孙奇锋、王麒诚和中信建投-数贝泽华人工智...


In [19]:
final.to_csv('./data/tidyed_data.csv', index=False, encoding='utf_8_sig')